In [2]:
#stuck using colab for experiments for now- oregon is broke :(
import torch
from tape import ProteinBertModel, TAPETokenizer
#unirep
from tape import UniRepModel
#getting the sweet, sweet < 2K AA proteins to use for the TAPE embeddings
import string
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
import re
from urllib.request import urlopen
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

from io import TextIOWrapper
import time
import os.path
import matplotlib.pyplot as plt
from os import path
from queue import PriorityQueue
from itertools import combinations
from numpy.random import choice
import random
import sqlite3

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [3]:
#open the csv
fragoza_seqs = pd.read_csv( "FragozaSequencesMaxLen2000.tab", sep = "\t", header = None)
print (fragoza_seqs)

                 0                                                  1
0      84069_R539P  MGNSHCVPQAPRRLRASFSRKPSLKGNREDSARMSAGLPGPEAARS...
1            84069  MGNSHCVPQAPRRLRASFSRKPSLKGNREDSARMSAGLPGPEAARS...
2      65220_R249W  MEMEQEKMTMNKELSPDAAAYCCSACHGDETWSYNHPIRGRAKSRS...
3            65220  MEMEQEKMTMNKELSPDAAAYCCSACHGDETWSYNHPIRGRAKSRS...
4     339448_V195M  MRSRKLTGAVRSSARLKARSCSAARLASAQEVAGSTSAKTACLTSS...
5           339448  MRSRKLTGAVRSSARLKARSCSAARLASAQEVAGSTSAKTACLTSS...
6     339448_R131C  MRSRKLTGAVRSSARLKARSCSAARLASAQEVAGSTSAKTACLTSS...
7      339448_S82A  MRSRKLTGAVRSSARLKARSCSAARLASAQEVAGSTSAKTACLTSS...
8      339448_A81T  MRSRKLTGAVRSSARLKARSCSAARLASAQEVAGSTSAKTACLTSS...
9       64802_A13T  MENSEKTEVVLLTCGSFNPITNMHLRLFELAKDYMNGTGRYTVVKG...
10           64802  MENSEKTEVVLLACGSFNPITNMHLRLFELAKDYMNGTGRYTVVKG...
11      64802_R66W  MENSEKTEVVLLACGSFNPITNMHLRLFELAKDYMNGTGRYTVVKG...
12      6723_V230M  MEPGPDGPAASGPAAIREGWFRETCSLWPGQALSLQVEQLLHHRRS...
13            6723  

In [5]:
unirep_model = UniRepModel.from_pretrained('babbler-1900')
unirep_model.cuda()
unirep_tokenizer = TAPETokenizer(vocab='unirep')

def getUniRepPooledSequence(sequence, model = unirep_model, tokenizer = unirep_tokenizer):
    #return
    #model = UniRepModel.from_pretrained('babbler-1900')
    #tokenizer = TAPETokenizer(vocab='unirep')
    token_ids = torch.tensor([tokenizer.encode(sequence)]).cuda()
    output = model(token_ids)
    return output[1]


def createReps2(table):
    for ind, row in table.iterrows():
      #for each wt and mt value 
      print ("On ", ind, " out of ", table.shape)
      name = row[0]
      seq = row[1]
      print("seq: ", name, " len seq: ", len(seq[1]))
      if not path.exists("./unirep_fragoza/" + str(name) + ".npy"):
        meanS1 = getUniRepPooledSequence(seq).cpu().detach().numpy()
        #print(meanS1.shape)
        #print (type(meanS1))
        #save npy file to the folder 
        np.save("./unirep_fragoza/" + str(name) + ".npy", meanS1)
        print ("bert done")


In [6]:
createReps2(fragoza_seqs)

On  0  out of  (3267, 2)
seq:  84069_R539P  len seq:  1


RuntimeError: CUDA out of memory. Tried to allocate 56.00 MiB (GPU 0; 11.17 GiB total capacity; 10.83 GiB already allocated; 28.25 MiB free; 10.83 GiB reserved in total by PyTorch)